# Определение тональности текста

## Вступление

Классификация тональности текста является важным инструментом для анализа общественного мнения, выявления позитивных и негативных настроений в текстах, например, в социальных сетях, отзывах пользователей или новостных статьях. 

В рамках данного проекта будет обучена модель логистической регрессии для классификации твитов на позитивные и негативные. Для этого будет использован набор данных с лемматизированными текстами твитов, где целевой признак — это столбец `positive`. Модель будет обучена на тренировочной выборке, а затем протестирована на тестовой выборке, для которой будет выполнено предсказание тональности текстов.

## План выполнения

1. **Загрузка данных:**
   - Загрузить тренировочную выборку из файла `tweets_lemm_train.csv` и тестовую выборку из `tweets_lemm_test.csv` с помощью библиотеки pandas.
   - Преобразовать столбец с текстами твитов в список строк.

2. **Предобработка данных:**
   - Извлечь из тренировочного набора целевой признак `positive`, который указывает на тональность текста (положительная или отрицательная).

3. **Векторизация текста:**
   - Для преобразования текста в числовые векторы использовать метод TF-IDF. Это поможет учесть как частоту появления слова в документе, так и его редкость в других документах.
   - Применить `TfidfVectorizer` из библиотеки `sklearn.feature_extraction.text` для извлечения признаков.

4. **Обучение модели:**
   - Обучить модель логистической регрессии на тренировочных данных с использованием библиотеки `sklearn.linear_model`.
   - Настроить гиперпараметры модели, если это необходимо, для улучшения точности.

5. **Оценка качества модели:**
   - Оценить качество модели с помощью метрики точности (accuracy) на тренировочной выборке, чтобы она не была ниже 0.62.
   - Использовать кросс-валидацию для проверки стабильности результатов.

6. **Предсказание на тестовой выборке:**
   - Использовать обученную модель для предсказания тональности на тестовой выборке (файл `tweets_lemm_test.csv`).
   - Сохранить результаты предсказания в новый файл с единственным столбцом `positive` (с нулями и единицами), используя метод `to_csv()` из pandas. Убедитесь, что файл не имеет расширения `.csv`, и в нем нет индексов.

7. **Проверка файла:**
   - Открыть полученный файл с результатами предсказания в текстовом редакторе и убедиться, что он содержит только один столбец с результатами (0 или 1).
   
8. **Заключение:**
   - Сделать выводы о качестве работы модели.

## Подготовка данных

### Загрузка

In [19]:
# Базовые библиотеки
import os, time, sklearn, warnings, nltk
import numpy as np
import pandas as pd
from datetime import datetime

# Модели и обработка данных
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Константы
RANDOM_STATE = 42

In [14]:
# Загрузка данных с учетом разделителей
train_path = 'sentiment/tweets_lemm_train.csv'
test_path = 'sentiment/tweets_lemm_test.csv'

# Загружаем список стоп-слов для русского языка
nltk.download('stopwords')
stop_words_russian = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nickolas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# Проверка наличия файлов и их загрузка
if os.path.exists(train_path):
    train_data = pd.read_csv(train_path)
    print(f'Тренировочный набор данных загружен успешно! {train_data.shape[0]} строк и {train_data.shape[1]} столбцов.')
else:
    print('Не удалось найти файл для тренировочного набора данных.')

if os.path.exists(test_path):
    test_data = pd.read_csv(test_path)
    print(f'Тестовый набор данных загружен успешно! {test_data.shape[0]} строк и {test_data.shape[1]} столбцов.')
else:
    print('Не удалось найти файл для тестового набора данных.')

Тренировочный набор данных загружен успешно! 5000 строк и 3 столбцов.
Тестовый набор данных загружен успешно! 3000 строк и 2 столбцов.


In [16]:
display(train_data.head())
display(test_data.head())

,text,positive,lemm_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,хоть я и школотый но поверь у мы то же самый о...
1,"Да, все-таки он немного похож на него. Но мой ...",1,да весь таки он немного похожий на он но мой м...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,ну ты идиотка я испугаться за ты
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,кто то в угол сидеть и погибать от голод а мы ...
4,@irina_dyshkant Вот что значит страшилка :D\r\...,1,вот что значит страшилка но блин посмотреть ве...


,text,lemm_text
0,RT @tiredfennel: если криса так интересуют дет...,если крис так интересовать ребёнок то либо они...
1,@xsealord по 200 руб. в месяц можно разместить...,по рубль в месяц можно разместить ссылка на те...
2,"@haosANDlaw @Etishkindyx учитывая, что сейчас ...",учитывать что сейчас преобладать один половина...
3,Товарищ :) Но я никак не могу отдельно не о...,товарищ но я никак не мочь отдельно не отметит...
4,RT @BodyaNick: Квн был отличный !) Оооочень по...,квн быть отличный оооочень понравиться женский...


In [17]:
# Извлекаем целевой признак
y_train = train_data['positive']

# Проверяем несколько примеров
print(train_data[['text', 'positive']].head())

                                                text  positive
0  @first_timee хоть я и школота, но поверь, у на...         1
1  Да, все-таки он немного похож на него. Но мой ...         1
2  RT @KatiaCheh: Ну ты идиотка) я испугалась за ...         1
3  RT @digger2912: "Кто то в углу сидит и погибае...         1
4  @irina_dyshkant Вот что значит страшилка :D\r\...         1


## Векторизация текста

In [20]:
# Подготовка данных (извлечение текста и целевого признака для обучения)
X_train = train_data['text']
y_train = train_data['positive']
X_test = test_data['text']

# Векторизация текста с использованием TF-IDF
vectorizer = TfidfVectorizer(stop_words=stop_words_russian)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

## Обучение модели

In [21]:
# Создание и обучение модели логистической регрессии
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

# Прогнозирование для тестовой выборки
y_pred = model.predict(X_test_tfidf)

# Сохраняем предсказания в новый DataFrame
predictions_df = pd.DataFrame({'positive': y_pred})

# Сохранение результатов в файл
predictions_df.to_csv('predictions', index=False)

# Печать первых 5 строк предсказаний
print(predictions_df.head())

# Вычисление точности модели (accuracy) на тестовой выборке
accuracy = accuracy_score(y_train, model.predict(X_train_tfidf))
print(f'Accuracy на обучающей выборке: {accuracy:.2f}')

   positive
0         1
1         0
2         0
3         0
4         1
Accuracy на обучающей выборке: 0.94


## Сохранение в файл

In [ ]:
# Создание DataFrame с предсказаниями
predictions_df = pd.DataFrame({'positive': y_pred})

# Сохранение предсказаний в файл без расширения и без индексов
predictions_df.to_csv('sentiment/predictions', index=False)

# Печать первых 5 строк результата для проверки
print(predictions_df.head())

   positive
0         1
1         0
2         0
3         0
4         1
